In [1]:
import pandas as pd
import numpy as np

In [2]:
BS = pd.read_parquet(r'../Resources/Section_26/SP500_balance_sheet.parquet')
IS = pd.read_parquet(r'../Resources/Section_26/SP500_income_statement.parquet')
companies = pd.read_parquet(r'../Resources/Section_26/SP500_constituents.parquet')

In [3]:
data = BS.merge(IS, on=['symbol', 'date'], how='inner', suffixes=['', 'is'], sort=True)

In [5]:
data[['symbol', 'companyName', 'date', 'fy', 'Equity', 'Assets', 'NI']].loc[data.symbol == 'IBM']

,symbol,companyName,date,fy,Equity,Assets,NI
2497,IBM,International Business Machines Corporation,2012-12-31,NaN,1.886000e+10,1.192130e+11,1.660400e+10
2498,IBM,International Business Machines Corporation,2013-12-31,2013.0,2.292900e+10,1.262230e+11,1.648300e+10
2499,IBM,International Business Machines Corporation,2014-12-31,2014.0,1.201400e+10,1.175320e+11,1.202200e+10
2500,IBM,International Business Machines Corporation,2015-12-31,2015.0,1.442400e+10,1.104950e+11,1.319000e+10
2501,IBM,International Business Machines Corporation,2016-12-31,2016.0,1.839200e+10,1.174700e+11,1.187200e+10
2502,IBM,International Business Machines Corporation,2017-12-31,2017.0,1.772500e+10,1.253560e+11,5.753000e+09
2503,IBM,International Business Machines Corporation,2018-12-31,2018.0,1.692900e+10,1.233820e+11,8.728000e+09
2504,IBM,International Business Machines Corporation,2019-12-31,2019.0,2.098500e+10,1.521860e+11,9.431000e+09
2505,IBM,International Business Machines Corporation,2020-12-31,2020.0,2.072700e+10,1.559710e+11,5.590000e+09
2506,IBM,International Business Machines Corporation,2021-12-31,2021.0,1.899600e+10,1.320010e+11,5.743000e+09


To calculate ROE correctly for 2013
ROE = 1.648300e+10 / ((1.886000e+10 + 2.292900e+10)/2)

And the same needs to be done for assets

ROE = This years NI divide by last years equity + this years ending equity

In [10]:
data[['Equity_lag1', 'Assets_lag1']] = data.groupby('symbol')[['Equity', 'Assets']].shift(1)

In [14]:
# add columns for last years equity and assets:

data[['Equity_lag1', 'Assets_lag1']]

,Equity_lag1,Assets_lag1
0,NaN,NaN
1,5.185000e+09,1.053600e+10
2,5.289000e+09,1.068600e+10
3,5.301000e+09,1.083100e+10
4,4.170000e+09,7.479000e+09
...,...,...
5428,1.786000e+09,8.586000e+09
5429,2.185000e+09,1.077700e+10
5430,2.708000e+09,1.154500e+10
5431,3.773000e+09,1.360900e+10


In [13]:
data[['symbol', 'companyName', 'date', 'fy', 'Equity', 'Equity_lag1', 'Assets', 'Assets_lag1', 'NI']].loc[data.symbol == 'IBM']

,symbol,companyName,date,fy,Equity,Equity_lag1,Assets,Assets_lag1,NI
2497,IBM,International Business Machines Corporation,2012-12-31,NaN,1.886000e+10,NaN,1.192130e+11,NaN,1.660400e+10
2498,IBM,International Business Machines Corporation,2013-12-31,2013.0,2.292900e+10,1.886000e+10,1.262230e+11,1.192130e+11,1.648300e+10
2499,IBM,International Business Machines Corporation,2014-12-31,2014.0,1.201400e+10,2.292900e+10,1.175320e+11,1.262230e+11,1.202200e+10
2500,IBM,International Business Machines Corporation,2015-12-31,2015.0,1.442400e+10,1.201400e+10,1.104950e+11,1.175320e+11,1.319000e+10
2501,IBM,International Business Machines Corporation,2016-12-31,2016.0,1.839200e+10,1.442400e+10,1.174700e+11,1.104950e+11,1.187200e+10
2502,IBM,International Business Machines Corporation,2017-12-31,2017.0,1.772500e+10,1.839200e+10,1.253560e+11,1.174700e+11,5.753000e+09
2503,IBM,International Business Machines Corporation,2018-12-31,2018.0,1.692900e+10,1.772500e+10,1.233820e+11,1.253560e+11,8.728000e+09
2504,IBM,International Business Machines Corporation,2019-12-31,2019.0,2.098500e+10,1.692900e+10,1.521860e+11,1.233820e+11,9.431000e+09
2505,IBM,International Business Machines Corporation,2020-12-31,2020.0,2.072700e+10,2.098500e+10,1.559710e+11,1.521860e+11,5.590000e+09
2506,IBM,International Business Machines Corporation,2021-12-31,2021.0,1.899600e+10,2.072700e+10,1.320010e+11,1.559710e+11,5.743000e+09


In [16]:
# Calculate the average:

data['Avg_Equity'] = ( data.Equity + data.Equity_lag1 ) / 2
data['Avg_Assets'] = ( data.Assets + data.Assets_lag1 ) / 2

In [19]:
data[['symbol', 'companyName', 'date', 'Equity', 'Equity_lag1', 'Avg_Equity', 'Assets', 'Assets_lag1', 'Avg_Assets', 'NI']].loc[data.symbol == 'IBM']

,symbol,companyName,date,Equity,Equity_lag1,Avg_Equity,Assets,Assets_lag1,Avg_Assets,NI
2497,IBM,International Business Machines Corporation,2012-12-31,1.886000e+10,NaN,NaN,1.192130e+11,NaN,NaN,1.660400e+10
2498,IBM,International Business Machines Corporation,2013-12-31,2.292900e+10,1.886000e+10,2.089450e+10,1.262230e+11,1.192130e+11,1.227180e+11,1.648300e+10
2499,IBM,International Business Machines Corporation,2014-12-31,1.201400e+10,2.292900e+10,1.747150e+10,1.175320e+11,1.262230e+11,1.218775e+11,1.202200e+10
2500,IBM,International Business Machines Corporation,2015-12-31,1.442400e+10,1.201400e+10,1.321900e+10,1.104950e+11,1.175320e+11,1.140135e+11,1.319000e+10
2501,IBM,International Business Machines Corporation,2016-12-31,1.839200e+10,1.442400e+10,1.640800e+10,1.174700e+11,1.104950e+11,1.139825e+11,1.187200e+10
2502,IBM,International Business Machines Corporation,2017-12-31,1.772500e+10,1.839200e+10,1.805850e+10,1.253560e+11,1.174700e+11,1.214130e+11,5.753000e+09
2503,IBM,International Business Machines Corporation,2018-12-31,1.692900e+10,1.772500e+10,1.732700e+10,1.233820e+11,1.253560e+11,1.243690e+11,8.728000e+09
2504,IBM,International Business Machines Corporation,2019-12-31,2.098500e+10,1.692900e+10,1.895700e+10,1.521860e+11,1.233820e+11,1.377840e+11,9.431000e+09
2505,IBM,International Business Machines Corporation,2020-12-31,2.072700e+10,2.098500e+10,2.085600e+10,1.559710e+11,1.521860e+11,1.540785e+11,5.590000e+09
2506,IBM,International Business Machines Corporation,2021-12-31,1.899600e+10,2.072700e+10,1.986150e+10,1.320010e+11,1.559710e+11,1.439860e+11,5.743000e+09


In [21]:
data['ROE'] = data.NI / data.Avg_Equity
data['ROA'] = data.NI / data.Avg_Assets

data.loc[data.Avg_Equity <= 0, 'ROE'] = np.nan
data.loc[data.Avg_Assets <= 0, 'ROA'] = np.nan

In [22]:
data[['symbol', 'companyName', 'date', 'fy', 'ROE', 'ROA']].loc[data.symbol == 'IBM']

,symbol,companyName,date,fy,ROE,ROA
2497,IBM,International Business Machines Corporation,2012-12-31,NaN,NaN,NaN
2498,IBM,International Business Machines Corporation,2013-12-31,2013.0,0.788868,0.134316
2499,IBM,International Business Machines Corporation,2014-12-31,2014.0,0.688092,0.098640
2500,IBM,International Business Machines Corporation,2015-12-31,2015.0,0.997806,0.115688
2501,IBM,International Business Machines Corporation,2016-12-31,2016.0,0.723549,0.104156
2502,IBM,International Business Machines Corporation,2017-12-31,2017.0,0.318576,0.047384
2503,IBM,International Business Machines Corporation,2018-12-31,2018.0,0.503723,0.070178
2504,IBM,International Business Machines Corporation,2019-12-31,2019.0,0.497494,0.068448
2505,IBM,International Business Machines Corporation,2020-12-31,2020.0,0.268028,0.036280
2506,IBM,International Business Machines Corporation,2021-12-31,2021.0,0.289152,0.039886


In [23]:
data[['symbol', 'companyName', 'date', 'fy', 'ROE', 'ROA']].loc[data.symbol == 'BA']

,symbol,companyName,date,fy,ROE,ROA
572,BA,The Boeing Company,2012-12-31,2012.0,NaN,NaN
573,BA,The Boeing Company,2013-12-31,2013.0,0.437417,0.050507
574,BA,The Boeing Company,2014-12-31,2014.0,0.457897,0.056770
575,BA,The Boeing Company,2015-12-31,2015.0,0.681636,0.053469
576,BA,The Boeing Company,2016-12-31,2016.0,1.345889,0.053090
577,BA,The Boeing Company,2017-12-31,2017.0,12.718386,0.089914
578,BA,The Boeing Company,2018-12-31,2018.0,25.450122,0.099765
579,BA,The Boeing Company,2019-12-31,2019.0,NaN,-0.005068
580,BA,The Boeing Company,2020-12-31,2020.0,NaN,-0.083097
581,BA,The Boeing Company,2021-12-31,2021.0,NaN,-0.028911


In [25]:
data[['ROE', 'ROA']].describe()

,ROE,ROA
count,4750.000000,4933.000000
mean,0.636268,0.085279
std,10.366045,0.847611
min,-248.500000,-0.618205
25%,0.084732,0.024563
50%,0.150196,0.056716
75%,0.266407,0.103571
max,441.577409,49.967970
